In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [2]:
params_dict_mu = {
    'input_size': 3,
    'hidden_size': 10,
    'output_size': 3,
    'ode_nn': ((50, "tanh"), (50, "tanh")),
    'readout_nn': ((50, "tanh"), (50, "tanh")),
    'enc_nn': ((50, "tanh"), (50, "tanh")),
    'use_rnn': False,
    'options': {'which_loss': 'easy'},
    "input_coords": np.arange(3),
    "output_coords": np.arange(3)
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from models import get_ckpt_model, NJODE
model_mu = NJODE(**params_dict_mu).to(device)
optimizer = torch.optim.Adam(model_mu.parameters())
get_ckpt_model("../data/saved_models/id-51/last_checkpoint/", model_mu, optimizer, device)
model_mu.eval()

using loss: easy
neuralODE use input scaling with tanh
use residual network: input_size=3, output_size=10
use residual network: input_size=10, output_size=3


NJODE(
  (ode_f): ODEFunc(
    (f): Sequential(
      (0): Linear(in_features=15, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
      (6): Linear(in_features=50, out_features=10, bias=True)
    )
  )
  (encoder_map): FFNN(
    (ffnn): Sequential(
      (0): Linear(in_features=3, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
      (6): Linear(in_features=50, out_features=10, bias=True)
    )
  )
  (readout_map): FFNN(
    (ffnn): Sequential(
      (0): Linear(in_features=10, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
 

In [3]:
params_dict_vol = {
    'input_size': 9,
    'hidden_size': 10,
    'output_size': 9,
    'ode_nn': ((50, "tanh"), (50, "tanh")),
    'readout_nn': ((50, "tanh"), (50, "tanh")),
    'enc_nn': ((50, "tanh"), (50, "tanh")),
    'use_rnn': False,
    'options': {'which_loss': 'easy_vol'},
    "input_coords": np.arange(9),
    "output_coords": np.arange(9)
}
model_vol = NJODE(**params_dict_vol).to(device)
optimizer_vol = torch.optim.Adam(model_vol.parameters())
get_ckpt_model("../data/saved_models/id-52/last_checkpoint/", model_vol, optimizer_vol, device)
model_vol.eval()

using loss: easy_vol
neuralODE use input scaling with tanh
use residual network: input_size=9, output_size=10
use residual network: input_size=10, output_size=9


NJODE(
  (ode_f): ODEFunc(
    (f): Sequential(
      (0): Linear(in_features=21, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
      (6): Linear(in_features=50, out_features=10, bias=True)
    )
  )
  (encoder_map): FFNN(
    (ffnn): Sequential(
      (0): Linear(in_features=9, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
      (6): Linear(in_features=50, out_features=10, bias=True)
    )
  )
  (readout_map): FFNN(
    (ffnn): Sequential(
      (0): Linear(in_features=10, out_features=50, bias=True)
      (1): Tanh()
      (2): Dropout(p=0, inplace=False)
      (3): Linear(in_features=50, out_features=50, bias=True)
      (4): Tanh()
      (5): Dropout(p=0, inplace=False)
 

In [4]:
mu_data=np.load("../data/training_data/OrnsteinUhlenbeckMultiDimensional-46/data.npy")

In [5]:
 def generate_next_value( X_t, mu_t, sigma_t, delta_t):
    """
    Generate the next value in the time series using the Euler-Maruyama scheme.

    :param X_t: current value tensor of shape (batch_size, d)
    :param mu_t: drift coefficient tensor of shape (batch_size, d)
    :param sigma_t: diffusion coefficient tensor of shape ( d, d)
    :param delta_t: time difference float
    :return: next value tensor of shape (batch_size, d)
    """
    delta_Wt = torch.randn_like(X_t) * delta_t** 0.5

    X_t_next = X_t + mu_t * delta_t + delta_Wt @ sigma_t

    return X_t_next

In [26]:
times=np.array([0.01,])
time_ptr = np.array([0, 1])
obs_idx = torch.tensor([0],dtype=torch.long)
Vol = torch.tensor([],dtype=torch.float)
delta_t=0.01
X= torch.tensor(mu_data[0,:,0], dtype=torch.float).unsqueeze(0)
start_X = torch.tensor(mu_data[0,:,0], dtype=torch.float).unsqueeze(0)
n_obs_ot=torch.tensor([1], dtype=torch.float)

for i in range(2,10):
    times=np.append(times, 0.01*i)
    time_ptr = np.append(time_ptr, i) 
    T=0.01*i
    obs_idx=torch.cat((obs_idx, torch.tensor([0], dtype=torch.long)))
    n_obs_ot+=1
    X= torch.cat((X, torch.tensor(mu_data[0,:,i-1], dtype=torch.float).unsqueeze(0)))
    
    # Here mu_pred is conditional expectation of X_t at time 0.01*(i-1) 
    mu_pred=model_mu.get_pred(times, time_ptr, X, obs_idx, delta_t, T, start_X, n_obs_ot)["pred"][-1]

    X_t_plus = X[i-1,:].unsqueeze(0)
    X_t = X[i-2, :].unsqueeze(0)
    vol_tensor = (X_t_plus - X_t).T@(X_t_plus-X_t)
    if i==2:
        start_Vol = vol_tensor.view(1,-1)
    vol  = vol_tensor.view(1,-1)
    Vol = torch.cat((Vol, vol))

    # Here sigma_pred conditional expectation of Z_t at time 0.01*(i-2) 
    sigma_pred=model_vol.get_pred(times[:-1], time_ptr[:-1], Vol, obs_idx[:-1], delta_t, T-0.01, start_Vol, (n_obs_ot-1) )["pred"][-1]
    
    # Computed mu_hat and sigma_hat at time 0.01*(i-2) 
    mu_hat_t = (mu_pred - X_t)/delta_t
    sigma_hat_t = sigma_pred/(delta_t**0.5)

    # generated new point at time 0.01*(i-1) 
    print(X_t, mu_hat_t*delta_t, torch.randn_like(X_t) @ sigma_hat_t.view(3,3)*(delta_t**0.5))
    
    X_next = generate_next_value(X_t, mu_hat_t, sigma_hat_t.view(3,3), delta_t)
    
    # Replace the old real data at time 0.01*(i-1) with generated data 
    X[i-1,:] = X_next


tensor([[1.0000, 1.5000, 2.0000]]) tensor([[0.0497, 0.0355, 0.0486]], grad_fn=<MulBackward0>) tensor([[0.0366, 0.0264, 0.0244]], grad_fn=<MulBackward0>)
tensor([[1.0544, 1.5460, 2.0436]], grad_fn=<UnsqueezeBackward0>) tensor([[0.0495, 0.0467, 0.0141]], grad_fn=<MulBackward0>) tensor([[-0.0057,  0.0063,  0.0344]], grad_fn=<MulBackward0>)
tensor([[1.1516, 1.6665, 2.1265]], grad_fn=<UnsqueezeBackward0>) tensor([[-0.0309, -0.0715, -0.0668]], grad_fn=<MulBackward0>) tensor([[-0.0076,  0.0034, -0.0181]], grad_fn=<MulBackward0>)
tensor([[1.1649, 1.6443, 2.1025]], grad_fn=<UnsqueezeBackward0>) tensor([[-0.0198, -0.0294,  0.0036]], grad_fn=<MulBackward0>) tensor([[-0.0059, -0.0120,  0.0007]], grad_fn=<MulBackward0>)
tensor([[1.1432, 1.6000, 2.0806]], grad_fn=<UnsqueezeBackward0>) tensor([[0.0229, 0.0275, 0.0447]], grad_fn=<MulBackward0>) tensor([[-0.0116, -0.0201,  0.0004]], grad_fn=<MulBackward0>)
tensor([[1.1417, 1.6389, 2.1173]], grad_fn=<UnsqueezeBackward0>) tensor([[0.0441, 0.0247, 0.0169]

In [21]:
X

tensor([[1.0000, 1.5000, 2.0000],
        [0.9628, 1.4408, 2.1674],
        [1.6178, 2.0943, 2.0169],
        [0.2142, 0.6047, 2.1017]], grad_fn=<CopySlices>)